In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import scipy.stats as sc
#import shap
#import lime
import sklearn 
import warnings
#import xgboost

import multiprocessing as mp
import time
import pickle 
#import interpret
import collections
import math
import random
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import  RandomForestRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn import preprocessing
#from interpret.glassbox import ExplainableBoostingClassifier
#from interpret import show
from sklearn.preprocessing import LabelEncoder
#from interpret.glassbox import ExplainableBoostingRegressor
#from interpret import show
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostRegressor
%matplotlib inline
import os, sys
#import statsmodels.api as sm
sys.path.append(os.path.abspath("../../../"))

pd.set_option('display.max_columns', None)
#import eli5
#from eli5.sklearn import PermutationImportance


In [ ]:
test_final_dist = pd.read_csv("test_final_dist.csv")
full_data_test = pd.read_csv("full_data_test.csv")

In [ ]:
full_data_test_no_clust = full_data_test[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Deck"]]
full_data_test_no_clust

In [ ]:
X = full_data_test[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Deck"]]
y = full_data_test[["clust"]]

In [ ]:
from sklearn import tree

In [ ]:
dt_feature_names = list(X.columns)
dt_target_names = ["0", "1", "2"]
clf = DecisionTreeClassifier(max_depth=3).fit(X, y)
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf, 
                   feature_names=dt_feature_names,  
                   class_names=dt_target_names,
                   filled=True)

In [ ]:
# Split a dataset based on an attribute and an attribute value
def test_split(feature, value, df):
    left, right = list(), list()
    indleft, indright = list(), list()
    for index, row in df.iterrows():
        if row[feature] < value:
            left.append(row)
            indleft.append(index)
        else:
            right.append(row)
            indright.append(index)
    return left, right, indleft, indright
 
# Calculate the distance index for a split dataset
def distance_index(left, right, indleft, indright):
    mean_left_dist = 0
    mean_right_dist = 0
    for i in range(len(indleft)-1):
        nodei_df = test_final_dist[test_final_dist['i1'] == i]
        nodei_j_df = nodei_df[nodei_df['i2'] == i+1]
        dist = test_final_dist["tree_dist"][0]
        mean_left_dist += dist 
    for i in range(len(indright)-1):
        nodei_df = test_final_dist[test_final_dist['i1'] == i]
        nodei_j_df = nodei_df[nodei_df['i2'] == i+1]
        dist = test_final_dist["tree_dist"][0]
        mean_right_dist += dist
    if len(left) == 0 and len(right) == 0:
        return 0
    elif len(left) == 0:
        mean_left_dist = 0
        mean_right_dist = mean_right_dist / len(indright)
        return mean_left_dist * len(left) / (len(left) + len(right)) + mean_right_dist * len(right) / (len(left) + len(right))
    elif len(right) == 0:
        mean_left_dist = mean_left_dist / len(indleft)
        mean_right_dist = 0
        return mean_left_dist * len(left) / (len(left) + len(right)) + mean_right_dist * len(right) / (len(left) + len(right))
    else:
        mean_left_dist = mean_left_dist / len(indleft)
        mean_right_dist = mean_right_dist / len(indright)
        return mean_left_dist * len(left) / (len(left) + len(right)) + mean_right_dist * len(right) / (len(left) + len(right))
        

In [ ]:
# Select the best split point for a dataset
def get_split(df):
    class_values = df["Survived"]
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for feature in df.columns:
        if feature == 'Survived':
            pass
        else:
            for index, row in df.iterrows():
                left, right, indleft, indright = test_split(feature, row[feature], df)
                dist = distance_index(left, right, indleft, indright)
                groups = left, right, indleft, indright
                if dist < b_score:
                    b_index, b_value, b_score, b_groups = feature, row[feature], dist, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


# Create a terminal node value (group means left or right)
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)
        
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[%s < %.3f]' % ((depth*' ', (node['index']), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [ ]:
dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,0],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,1]]

In [ ]:
df = pd.DataFrame(dataset, columns = ['X1', 'X2', 'class'])

In [ ]:
df

In [ ]:
full_data_test_no_clust.columns

In [ ]:
small_data_test_no_clust = full_data_test_no_clust.head(25)
small_data_test_no_clust["Survived"]

In [ ]:
tree = build_tree(small_data_test_no_clust, 2, 1)
tree

In [ ]:
print_tree(tree)

In [ ]:

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right
 
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini
 
# Select the best split point for a dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}
 
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

In [ ]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root
 
# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print(type(node['index']+1))
        print(type(node['value']))
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [ ]:
tree = build_tree(dataset, 3, 1)
print_tree(tree)

In [ ]:
def foo(x):
    if x == 3: 
        continue
    if x % 3 == 0:
        print("hi")
    else:
        print('no')
foo(3)